In [ ]:
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch] -q

In [ ]:
import os
import torch
from transformers import get_linear_schedule_with_warmup, TrainingArguments, Trainer
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader, random_split
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, get_peft_model_state_dict, prepare_model_for_int8_training, set_peft_model_state_dict
from transformers import AdamW, get_scheduler,TrainerCallback
from sklearn.model_selection import train_test_split

import torch.nn as nn

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from typing import List
from transformers import DataCollatorForSeq2Seq

In [ ]:
max_length = 512

In [ ]:
base_model = 'google/flan-t5-base'

In [ ]:
batch_size = 6

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Загрузка данных из CSV файла
df = pd.read_csv('/content/drive/MyDrive/МФТИ/nlp2/chatbot/Data/Barney2.csv')
df.head(5)

,name,text,context
0,Barney,"(on the phone) hey, so you know how I've alwa...",What was I doing? Your Uncle Marshall was tak...
1,Barney,"Okay, meet me at the bar in fifteen minutes, ...","Hey, you wanna do something tonight? Hey."
2,Barney,Where's your suit!? Just once when I say suit...,Hey. I did that one time.
3,Barney,It was a blazer!,"I did that one time. You know, ever since co..."
4,Barney,I see what this is about. Have you forgotten ...,"You know, ever since college it's been Marsha..."


In [ ]:
# Инициализация токенизатора
tokenizer = T5Tokenizer.from_pretrained(base_model)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = dataframe.dropna()  # Удаление строк с NaN значениями

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx]['text']) + str(self.data.iloc[idx]['context'])
        encoding = self.tokenizer(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        input_ids = encoding.input_ids.squeeze(0)
        labels = input_ids.clone()  # Создаем копию input_ids
        labels[:-1] = input_ids[1:]  # Сдвигаем на один токен вперед
        labels[-1] = self.tokenizer.eos_token_id  # Заменяем последний токен на токен окончания последовательности
        return {"input_ids": input_ids, "labels": labels}


In [ ]:
dataset = CustomDataset(df, tokenizer, max_length)

In [ ]:
dataset[0]

{'input_ids': tensor([   41,   106,     8,   951,    61,     3, 13133,     6,    78,    25,
           214,   149,    27,    31,   162,   373,   141,     3,     9,   589,
            21,   985,    18,   188, 10488,  3567,    58,  1548,     6,   230,
            27,    31,   162,   530,     3,     9,   126,  1305,    10,   312,
          3478,    15,     7,    15,  3567,    55,   312,  3478,    15,     7,
            15,  3567,    33,     8,   126,   985,    18,   188,     7,  7137,
             5,   363,    47,    27,   692,    58,   696, 26706, 17410,    47,
           838,     8,  2630,  1147,    13,   112,   280,     6,    11,   140,
            18,   196,    31,    51,  3874,    39, 26706,     6,  1386,  3186,
             5,  9459,     6,    25, 14371,     9,   103,   424,  8988,    58,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [ ]:
def resume_training(model, resume_from_checkpoint):
    if resume_from_checkpoint:
        # Проверяем наличие доступных весов и загружаем их
        checkpoint_name = os.path.join(resume_from_checkpoint, "pytorch_model.bin")  # Полный чекпоинт
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(resume_from_checkpoint, "adapter_model.bin")  # только модель LoRA - конфигурация LoRA выше должна соответствовать
            resume_from_checkpoint = False  # чтобы тренер не пытался загрузить его состояние
        # Два вышеуказанных файла имеют разное имя в зависимости от того, как они были сохранены, но на самом деле они одинаковые.
        if os.path.exists(checkpoint_name):
            print(f"Продолжаем обучение с {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Чекпоинт {checkpoint_name} не найден")

    model.print_trainable_parameters()  # Более прозрачно отображаем % обучаемых параметров

In [ ]:
def train_t5(
    dataset: dataset,
    base_model: str = "google/flan-t5-base",
    max_length: int = 512,
    num_epochs: int = 4,
    batch_size: int = 8,
    learning_rate: float = 1e-6,
    resume_from_checkpoint: str = None,
    micro_batch_size: int = 4,
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
#    lora_target_modules: List[str] = ["q_proj", "v_proj"],
    train_on_inputs: bool = True,
    add_eos_token: bool = False,
    val_set_size: int = 0
):
    # Load tokenizer and model

    tokenizer = T5Tokenizer.from_pretrained(base_model)
    tokenizer.pad_token = tokenizer.eos_token

    model = T5ForConditionalGeneration.from_pretrained(base_model)

    # Apply any necessary model preparation steps
    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
 #       target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

    # Resume training if specified
    resume_training(model, resume_from_checkpoint)

    # Prepare data
    dataset = dataset
    train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

    # Prepare training arguments
    training_args = TrainingArguments(
        output_dir="/content/drive/MyDrive/МФТИ/nlp2/chatbot/generative/T5results",
        per_device_train_batch_size=micro_batch_size,
        gradient_accumulation_steps=batch_size // micro_batch_size,
        warmup_steps=100,
        evaluation_strategy="steps" if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=200 if val_set_size > 0 else None,
        save_steps=200,
        save_total_limit=3,
        load_best_model_at_end=True if val_set_size > 0 else False,
    )

    os.makedirs(training_args.output_dir, exist_ok=True)

    # Define Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=DataCollatorForSeq2Seq(
            tokenizer, return_tensors="pt", padding=True
        ),
      #  callbacks=callbacks()
    )

    # Disable model caching
    model.config.use_cache = False

    # Save original state dictionary
    old_state_dict = model.state_dict()

    # Replace state_dict method with custom function
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict
        )
    ).__get__(model, type(model))

    # Train the model
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    # Save the trained model
    model.save_pretrained(training_args.output_dir)

    print("Training complete.")

        # Plot losses
    train_losses = train_result.train_loss
    val_losses = train_result.eval_loss

    # Calculate moving average for train losses
    train_running_loss = [np.mean(train_losses[i:i+window_size])
                          for i in range(len(train_losses)-window_size)]

    # Calculate moving average for validation losses
    val_running_loss = [np.mean(val_losses[i:i+window_size])
                        for i in range(len(val_losses)-window_size)]

In [ ]:
# Example usage:
train_t5(dataset)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3560841867092814


Step,Training Loss
500,0.731500
1000,0.022700
1500,0.019900


Training complete.


NameError: name 'train_result' is not defined

In [ ]:
# Plotting
plt.figure(figsize=(10, 5))
plt.plot(train_running_loss, label='Train Loss')
plt.plot(val_running_loss, label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()